In [156]:
import base64
import hashlib

from json import loads as jl
from json import dumps as jd

import hmac

enc = "utf-8"

In [157]:
def byte2base64(x):
    return base64.urlsafe_b64encode(x).decode(enc)

def str2base64(x):
    return b2b64(x.encode(enc))

In [158]:
class User(object):
    def __init__(self, id, name, perm):
        self.id = id
        self.name = name
        self.perm = perm
        
    def to_json(self):
        return {
            "id": self.id, 
            "username": self.name, 
            "permission": self.perm
        }

In [197]:
import binascii

# jsonомешалка мешает бетон
# 
def dumps(j):
    return jd(j, separators=(',\r\n ', ':'))

def create_raw_header(alg, typ):
    return dumps({"typ": typ, "alg": alg})

def create_encoded_header(raw):
    return byte2base64(raw.encode(enc)).replace('=', '')

def create_raw_payload(data):
    return dumps(data)

def create_encoded_payload(raw):
    encoded = raw.encode(enc)
    return byte2base64(encoded).replace('=', '')

def encode_hmac(b, key):
    return byte2base64(hmac.new(
        key.encode(enc),
        msg=b,
        digestmod=hashlib.sha256
    ).digest()).replace('=', '')

In [ ]:
№

In [168]:
def massert(a, b):
    la = len(a)
    lb = len(b)
    if la < lb:
        raise AssertionError("a is shorter")
    if lb < la:
        raise AssertionError("b is shorter")
    for i in range(la):
        if a[i] != b[i]:
            raise AssertionError(f"missmatch on index {i} {a[i]} vs {b[i]}")

In [195]:
# tests

header_data = create_raw_header("HS256", "JWT")
header_hash = "eyJ0eXAiOiJKV1QiLA0KICJhbGciOiJIUzI1NiJ9"

payload_data = create_raw_payload({
    "iss":"joe",
    "exp":1300819380,
    "http://example.com/is_root":True
})
payload_hash = "eyJpc3MiOiJqb2UiLA0KICJleHAiOjEzMDA4MTkzODAsDQogImh0dHA6Ly9leGFtcGxlLmNvbS9pc19yb290Ijp0cnVlfQ"

massert(header_hash, create_encoded_header(header_data)) # header hash
massert(payload_hash, create_encoded_payload(payload_data)) # payload hash

In [196]:
# jwt.io test sign
# note: jwt.io uses no line separator in json notation.
# there's only , between lines
# so, hash values doesn't equals to rfc's ones
# but rfc unclearly explains sign phase
# so, there's a test to check how to sign those parts
# note: hmac(header . payload, secret) -> base64 with out '='
# p.s. sha256 used here

a = b"eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9"
b = b"eyJzdWIiOiIxMjM0NTY3ODkwIiwibmFtZSI6IkpvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQ"
ab = a + b"." + b

massert(encode_hmac(ab, '6741'), "8aRJfpBioIVCF43SXxRoVH_t756PGGtDavQgUXQ9PGU")